# Anomaly Detector Program and Uses

Here is the paper that was relevent to the development of the code:
Automatic Identification of Rainfall in Acoustic Recordings by 
Carol Bedoya et al. - https://www.researchgate.net/publication/312324083_Automatic_identification_of_rainfall_in_acoustic_recordings

Normally, rainfall would be like background noise, making it very hard to detect on a spectrogram. However, the power spectral density shows the energy (power) at a given frequency in a different format that allows us to notice whether the energy is weak or strong. On the 600-1200 Hz frequencies, there is a higher energy reading when there is rain, meaning we can interpret spikes in the 600-1200 Hz frequency as rain. However, it's still not enough to just use the power spectral density, as other animals or anthropogenic sources also emit sounds on that 600-1200 frequency. Thus, the signal to noise ratio is used to determine whether the sound is rainfall or something else. The signal to noise ratio is basically the ratio between a wanted sound to noise - the ratio was determined using complicated math you can read about in the paper. 

The code based off of the paper (anomaly_detector.py) is in the Github Repo. While the code is similar to rain_cal.py, there are some differences, but not very noticable. 

The dataset used to test and develop the code was gathered from Audiomoth devices in the Peruvian rainforest. It was recorded at a 384 kHz, a very high sampling rate. 

Please import the following libraries below:

In [1]:
from __future__ import division
import sys
import numpy as np
import matplotlib.pyplot as plt
import glob
import time
from scipy import signal
from scipy.io import wavfile
import scipy.signal as scipy_signal
# anamoly_detector is the .py file on our Github
import anomaly_detector

# Versions of Libraries that were used (most recent versions should work, but just in case) - any libraries not included are probably included in your system already:

future == 0.18.2

numpy == 1.18.5

matplotlib == 3.2.1

scipy == 1.4.1

# Files Included on Github

anomaly_detector.py - The main code that includes our classify and train function

rain1.wav - Rain and wind

rain2.wav - Rain and wind

talk.wav - People talking with a motor in the background

static.wav - Electrostatic noises interfering with cricket chirps

engine.wav - Car engine momentarily passing through with sputters caught

# Training

Will function at any frequency. However, you will have to change the frequency it functions at in the code. You can see that it was set to function between frequency 600-1200.

Basically, the train function will take in a set of "labeled" clips (in this case, rain) that you have put into a folder, like Rain_Datasets. It will go through and output the minimum and maximum Power Spectral Density and Signal to Noise ratio values for whatever type of noise you analyzed. These values are printed out for you to see and plugged into the Classify function.

In the below example, Rain_Datasets was used to get parameters to detect rain. The two clips used in Rain_Datasets was rain1 and rain2. It then classified the other clips in Mixed_AM_Datasets (including rain1 and rain2, sorry) to detect rain/wind vs. human noises/electrostatic. 

In [2]:
	### Set True or False depending on whether you want to train or use default thresholds ###
	TRAIN = True

	### Set your own file path ###
	train_path = 'Rain_Datasets/'
	classify_path = 'Mixed_AM_Datasets/'
    
	if TRAIN:
		start_time = time.time()
		min_frq, max_frq = 600, 1200
		min_psd, max_psd, min_snr, max_snr = anomaly_detector.train(train_path)
		print('min_psd: '+ str(min_psd)) 
		print('max_psd: '+ str(max_psd))
		print('min_snr: '+ str(min_snr))
		print('max_snr: '+ str(max_snr))
		print('----- Training: {:.2f} seconds -----'.format(
				time.time() - start_time))
	else:
		min_psd, max_psd, min_snr, max_snr, min_frq, max_frq = 1e-6, 2, 3.5, sys.maxsize, 600, 1200

	start_time = time.time()
	anomaly_detector.classify(classify_path, min_psd, max_psd, min_snr, max_snr, min_frq, max_frq)
	print('----- Classification: {:.2f} seconds -----'.format(
			time.time() - start_time))

min_psd: 1.2029623557037434
max_psd: 1.6864378627214969
min_snr: 19.549865511143132
max_snr: 45.22966212908287
----- Training: 6.32 seconds -----
----- Wind or Rain -----
Mixed_AM_Datasets\rain1.WAV: Intensity 1.20
Mixed_AM_Datasets\rain2.WAV: Intensity 1.69
----- Human Noises or Electrostatic -----
Mixed_AM_Datasets\talk.WAV: Intensity 3.54
----- Classification: 16.78 seconds -----


# Classification: 600 - 1200 Hz Detection System

Functions between the 600 - 1200 Hz frequency range. Will classify clips as either rain/wind or human noises/electrostatic. This divide is based on the intensity of a clip. A clip that has an intensity of 2 or below is classified as rain/wind. One that has an intensity higher than 2 is considered human noises/electrostatic. 

Below the "else" in the below code, there is a set of values that are used by the Classify function to be able to classify the clips. Modify them for your own purpose. 

In [3]:
	### Set True or False depending on whether you want to train or use default thresholds ###
	TRAIN = False

	### Set your own file path ###
	train_path = 'Rain_Datasets/'
	classify_path = 'Mixed_AM_Datasets/'
    
	if TRAIN:
		start_time = time.time()
		min_frq, max_frq = 600, 1200
		min_psd, max_psd, min_snr, max_snr = anomaly_detector.train(train_path)
		print('min_psd: '+ str(min_psd)) 
		print('max_psd: '+ str(max_psd))
		print('min_snr: '+ str(min_snr))
		print('max_snr: '+ str(max_snr))
		print('----- Training: {:.2f} seconds -----'.format(
				time.time() - start_time))
	else:
		min_psd, max_psd, min_snr, max_snr, min_frq, max_frq = 1e-6, 2, 3.5, sys.maxsize, 600, 1200

	start_time = time.time()
	anomaly_detector.classify(classify_path, min_psd, max_psd, min_snr, max_snr, min_frq, max_frq)
	print('----- Classification: {:.2f} seconds -----'.format(
			time.time() - start_time))

----- Wind or Rain -----
Mixed_AM_Datasets\rain1.WAV: Intensity 1.20
Mixed_AM_Datasets\rain2.WAV: Intensity 1.69
----- Human Noises or Electrostatic -----
Mixed_AM_Datasets\engine.WAV: Intensity 3.61
Mixed_AM_Datasets\static.WAV: Intensity 2.28
Mixed_AM_Datasets\talk.WAV: Intensity 3.54
----- Classification: 16.19 seconds -----
